In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import json
import copy
import folium
import branca
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, fcluster

import panel as pn
import networkx as nx
import geopandas as gpd
import shapely
from shapely.geometry import LineString, MultiLineString, shape
import shapely.wkt
from matplotlib import cm, colors

## Grafo de conexion entre barrios

In [18]:
import panel as pn
import param
import geopandas as gpd
import pandas as pd
import shapely.wkt
import networkx as nx
import plotly.graph_objects as go

pn.extension('plotly')

def safe_load_geometry(wkt_str):
    try:
        return shapely.wkt.loads(wkt_str)
    except Exception:
        return None

class BusGraphPanel(param.Parameterized):
    barrio = param.ObjectSelector(default=None, objects=[])

    def __init__(self, **params):
        super().__init__(**params)

        # Cargar datos
        self.barrios_gdf = gpd.read_file('data/barris.geojson')
        bus_lines_df = pd.read_csv("data/linies_bus.csv")
        bus_lines_df["geometry"] = bus_lines_df["GEOMETRY"].apply(safe_load_geometry)
        self.bus_lines_gdf = gpd.GeoDataFrame(bus_lines_df, geometry="geometry", crs="EPSG:4326")

        # Crear grafo
        self.G = nx.Graph()
        for idx, row in self.barrios_gdf.iterrows():
            self.G.add_node(row["NOM"], geometry=row.geometry)

        for line in self.bus_lines_gdf.itertuples():
            if not line.geometry:
                continue
            touched_barrios = set()
            for idx, barrio in self.barrios_gdf.iterrows():
                if line.geometry.intersects(barrio.geometry):
                    touched_barrios.add(barrio["NOM"])
            touched_barrios = list(touched_barrios)
            for i in range(len(touched_barrios)):
                for j in range(i + 1, len(touched_barrios)):
                    b1, b2 = touched_barrios[i], touched_barrios[j]
                    if self.G.has_edge(b1, b2):
                        self.G[b1][b2]["weight"] += 1
                    else:
                        self.G.add_edge(b1, b2, weight=1)

        # Posiciones
        self.pos = {node: (geom.centroid.x, geom.centroid.y)
                    for node, geom in nx.get_node_attributes(self.G, 'geometry').items()}
        nx.set_node_attributes(self.G, self.pos, 'pos')

        self.node_data = [{'id': n, 'x': x, 'y': y} for n, (x, y) in self.pos.items()]
        self.param.barrio.objects = [n['id'] for n in self.node_data]

    @param.depends('barrio')
    def view(self):
        edge_traces = []
        for u, v, data in self.G.edges(data=True):
            x0, y0 = self.G.nodes[u]['pos']
            x1, y1 = self.G.nodes[v]['pos']
            edge_traces.append(go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                line=dict(width=data['weight'],
                          color='red' if self.barrio in (u, v) else 'lightgray'),
                mode='lines',
                opacity=1.0 if self.barrio in (u, v) else 0.2,
                hoverinfo='text',
                text=[f"{u} ↔ {v}: {data['weight']} líneas"] * 3,
                showlegend=False
            ))

        node_trace = go.Scatter(
            x=[n['x'] for n in self.node_data],
            y=[n['y'] for n in self.node_data],
            mode='markers+text',
            text=[n['id'] for n in self.node_data],
            textposition="top center",
            hoverinfo='text',
            marker=dict(size=10, color=[
                'orange' if n['id'] == self.barrio else 'skyblue' for n in self.node_data
            ], line_width=2),
            showlegend=False
        )

        fig = go.Figure(data=edge_traces + [node_trace])
        fig.update_layout(
            title="Conexiones entre barrios por líneas de bus",
            margin=dict(b=20, l=5, r=5, t=40),
            hovermode='closest',
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            width=700,      # fijo
            height=450      # fijo
        )

        return pn.pane.Plotly(
            fig,
            config={'responsive': False}, 
            sizing_mode='fixed',
            width=700,
            height=450
        )

# Función para usar como tab en el dashboard
def crear_tab_conexiones_bus():
    app = BusGraphPanel()

    # 1) Selector de barrio (ancho fijo, no compite por espacio)
    controles = pn.WidgetBox(
        pn.pane.Markdown("## Barrio"),
        app.param.barrio,
        width=300,
        sizing_mode='fixed'
    )
    controles.flex = 0

    # 2) Pane reactivo: le pasamos la función, no su resultado
    grafico_pane = pn.panel(
        app.view,                # <- función @param.depends
        sizing_mode='fixed',     # tamaño fijo para no variar
        width=700,
        height=450,
        config={'responsive': False}
    )
    # Le decimos que siempre ocupe su 700px y reevalúe
    grafico_pane.flex = 0

    # 3) Fila con selector y gráfico
    fila = pn.Row(
        controles,
        grafico_pane,
        sizing_mode='stretch_width'
    )

    # 4) Columna contenedora de la pestaña
    return pn.Column(
        pn.pane.Markdown("## Conexiones entre barrios por líneas de bus"),
        fila,
        sizing_mode='stretch_width'
    )


/* Override VSCode background color */
.cell-output-ipywidget-background:has(
 > .cell-output-ipywidget-background > .lm-Widget > *[data-root-id]
 ),
.cell-output-ipywidget-background:has(> .lm-Widget > *[data-root-id]) {
 background-color: transparent !important;
}

## Evolucion temporal desempleo

In [33]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

# Cargar datos
df = pd.read_csv("data/unemployment.csv")
df["Date"] = pd.to_datetime(df["Year"].astype(str) + "-" + df["Month"], format="%Y-%B")

# Agregar desempleo total por género, barrio y fecha
agg_df = df.groupby(["Date", "Neighborhood Name", "Gender"])["Number"].sum().reset_index()

# Listado de barrios
neighborhoods = sorted(agg_df["Neighborhood Name"].unique().tolist())

# Selector múltiple de barrios
selector = widgets.SelectMultiple(
    options=neighborhoods,
    value=["el Raval"],
    description='Barrios',
    rows=12,
    layout=widgets.Layout(width='50%')
)

# Paleta de colores por barrio (usamos px.colors.qualitative.Plotly)
palette = px.colors.qualitative.Plotly
barrio_colors = {barrio: palette[i % len(palette)] for i, barrio in enumerate(neighborhoods)}

# Función de graficado
def plot_unemployment(selected_neighs):
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
                        subplot_titles=("Desempleo Masculino", "Desempleo Femenino"))

    for barrio in selected_neighs:
        color = barrio_colors[barrio]
        male_series = agg_df[(agg_df["Neighborhood Name"] == barrio) & (agg_df["Gender"] == "Male")]
        female_series = agg_df[(agg_df["Neighborhood Name"] == barrio) & (agg_df["Gender"] == "Female")]

        fig.add_trace(
            go.Scatter(x=male_series["Date"], y=male_series["Number"], name=barrio,
                       mode='lines', line=dict(color=color)),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=female_series["Date"], y=female_series["Number"], name=barrio,
                       mode='lines', line=dict(color=color), showlegend=False),
            row=1, col=2
        )

    fig.update_layout(height=500, width=1000, title_text="Evolución del desempleo por género")
    fig.update_xaxes(title_text="Fecha", row=1, col=1)
    fig.update_xaxes(title_text="Fecha", row=1, col=2)
    fig.update_yaxes(title_text="Número de desempleados", row=1, col=1)

    fig.show()

# Conexión del widget con la función
widgets.interact(plot_unemployment, selected_neighs=selector);


interactive(children=(SelectMultiple(description='Barrios', index=(46,), layout=Layout(width='50%'), options=(…

## Air Quality

In [41]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Recarga del archivo (después de reinicio, suponiendo que ha sido subido como 'air_quality.csv')
df = pd.read_csv("data/air_quality_Nov2017.csv")

# Conversión de columnas a numérico
for col in ["O3 Value", "NO2 Value", "PM10 Value"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Función única que genera las tres gráficas como subplots
def plot_all_histograms(df):
    fig = make_subplots(rows=1, cols=3, subplot_titles=["Distribución de O3", "Distribución de NO2", "Distribución de PM10"])

    contaminants = [
        ("O3 Value", "O3 Quality", "O3", 1),
        ("NO2 Value", "NO2 Quality", "NO2", 2),
        ("PM10 Value", "PM10 Quality", "PM10", 3)
    ]

    for value_col, quality_col, title, col_index in contaminants:
        # Histograma
        fig.add_trace(go.Histogram(
            x=df[value_col].dropna(),
            nbinsx=30,
            marker_color='steelblue',
            name=title,
            showlegend=False
        ), row=1, col=col_index)

        # Fondos por calidad
        good_vals = df[df[quality_col] == "Good"][value_col].dropna()
        mod_vals = df[df[quality_col] == "Moderate"][value_col].dropna()

        if not good_vals.empty:
            fig.add_vrect(
                x0=good_vals.min(), x1=good_vals.max(),
                fillcolor="green", opacity=0.2, layer="below", line_width=0,
                row=1, col=col_index
            )
        if not mod_vals.empty:
            fig.add_vrect(
                x0=mod_vals.min(), x1=mod_vals.max(),
                fillcolor="orange", opacity=0.2, layer="below", line_width=0,
                row=1, col=col_index
            )

    fig.update_layout(
        height=500,
        width=1200,
        title_text="Histogramas de contaminantes",
        bargap=0.1
    )
    fig.update_xaxes(title_text="Valor")
    fig.update_yaxes(title_text="Frecuencia")
    fig.show()

# Ejecutar la función
plot_all_histograms(df)
